In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords

data = pd.read_csv('data/train.csv')

stops = stopwords.words('english')

vectorizer = TfidfVectorizer(max_df=0.95, min_df=5, stop_words=stops, ngram_range=(1, 3), max_features=10000)

In [2]:
import numpy as np

def accuracy(y_pred, y):
    tp = np.sum(np.logical_and(y_pred == 1, y == 1))
    tn = np.sum(np.logical_and(y_pred == 0, y == 0))
    fp = np.sum(np.logical_and(y_pred == 1, y == 0))
    fn = np.sum(np.logical_and(y_pred == 0, y == 1))
    
    return tp, tn, fp, fn

y_pred = np.asarray([1, 1, 0])
y = np.asarray([1, 0, 1])

accuracy(y_pred, y)

(1, 0, 1, 1)

In [47]:
from sklearn.model_selection import train_test_split

X = vectorizer.fit_transform(data.comment_text)
ids = data.id.values

y_layer_1 = data.toxic.values
y_layer_2 = data[['severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].values

test_split = 0.2
dev_split = 0.1
random_state = 42

X_train, X_test, y_train_1, y_test_1, y_train_2, y_test_2, id_train, id_test = train_test_split(
    X,
    y_layer_1, 
    y_layer_2,
    ids,
    test_size = 1 - (test_split + dev_split),
    random_state = random_state
)

X_dev, X_test, y_dev_1, y_test_1, y_dev_2, y_test_2, id_dev, id_test = train_test_split(
    X_test,
    y_test_1,
    y_test_2,
    id_test,
    test_size = test_split / (test_split + dev_split),
    random_state = random_state
)

In [48]:
from classification_utils import *
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier

layer_1 = LogisticRegression(C=3, solver='liblinear', max_iter=10000, class_weight='balanced')
layer_2_estimator = LogisticRegression(C=6, solver='liblinear', max_iter=10000, class_weight='balanced')
layer_2 = MultiOutputClassifier(layer_2_estimator)
layer_1.fit(X_train, y_train_1)
y_pred_layer_1 = layer_1.predict(X_test)
print("Layer 1 score: ", layer_1.score(X_test, y_test_1))

Layer 1 score:  0.9342527562544483


In [49]:
print(y_train_2[y_train_1 == 1].shape)
print(X_train[y_train_1 == 1].shape)

layer_2.fit(X_train[y_train_1 == 1], y_train_2[y_train_1 == 1])
print("Layer 2 train score: ", layer_2.score(X_train[y_train_1 == 1], y_train_2[y_train_1 == 1]))
y_pred_layer_2 = layer_2.predict(X_test[y_pred_layer_1 == 1])
print("Layer 2 test score: ", layer_2.score(X_test[y_pred_layer_1 == 1], y_test_2[y_pred_layer_1 == 1]))

(4636, 5)
(4636, 10000)
Layer 2 train score:  0.7758843830888698
Layer 2 test score:  0.5421249332621463


In [69]:
y_pred = np.zeros((y_pred_layer_1.shape[0], 6))
y_pred[:, 0] = y_pred_layer_1
y_pred[y_pred_layer_1 == 1, 1:] = y_pred_layer_2

y = np.hstack((y_test_1.reshape(-1, 1), y_test_2))

from sklearn.metrics import classification_report, accuracy_score

print('Accuracy: {:.2f}%'.format(accuracy_score(y, y_pred) * 100))
print(classification_report(y, y_pred, target_names= ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']))

Accuracy: 88.54%
               precision    recall  f1-score   support

        toxic       0.62      0.82      0.70      7083
 severe_toxic       0.35      0.58      0.44       761
      obscene       0.83      0.72      0.77      3935
       threat       0.35      0.51      0.42       222
       insult       0.67      0.63      0.65      3718
identity_hate       0.37      0.50      0.43       678

    micro avg       0.63      0.72      0.67     16397
    macro avg       0.53      0.63      0.57     16397
 weighted avg       0.65      0.72      0.68     16397
  samples avg       0.07      0.07      0.06     16397



In [71]:
y_train = np.hstack((y_train_1.reshape(-1, 1), y_train_2))
y_test = np.hstack((y_test_1.reshape(-1, 1), y_test_2))

layer_2.fit(X_train, y_train)
y_pred = layer_2.predict(X_test)

from sklearn.metrics import classification_report, accuracy_score

print('Accuracy: {:.2f}%'.format(accuracy_score(y_test, y_pred) * 100))
print(classification_report(y_test, y_pred, target_names= ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']))

Accuracy: 87.18%
               precision    recall  f1-score   support

        toxic       0.62      0.81      0.70      7083
 severe_toxic       0.29      0.73      0.42       761
      obscene       0.68      0.82      0.75      3935
       threat       0.26      0.60      0.36       222
       insult       0.55      0.78      0.65      3718
identity_hate       0.27      0.60      0.37       678

    micro avg       0.56      0.79      0.65     16397
    macro avg       0.44      0.72      0.54     16397
 weighted avg       0.58      0.79      0.67     16397
  samples avg       0.06      0.07      0.06     16397



C:\Users\Jacob Beel\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Jacob Beel\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
